In [1]:
qpath = r'C:/Users/rgoix/DEV/quetzal/' # path to quetzal here
data = r'inputs/'
import sys
sys.path.append(qpath + r'quetzal/')

# import class
from quetzal.io.gtfs_reader import importer

### Read GTFS

In [2]:
feed = importer.GtfsImporter(path=data + r'paris_ratp_full', dist_units='m')
feed.clean()
feed.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,"[12669243, 2707227, 2639291, 2676189, 12679709..."
2,timezone,CET
3,start_date,20181003
4,end_date,20190101
5,num_routes,1011
6,num_trips,447494
7,num_stops,25590
8,num_shapes,0
9,num_frequencies,0


In [3]:
feed.map_stops(feed.stops.stop_id)

### Restrict to given dates

This restricts the trips and associated data to those running in one of the given dates. This also restricts the feed to these dates only.

In [4]:
feed_r = feed.restrict(
    dates=['20181106', '20181111'] # the date must be within the feed start and end dates
)

In [5]:
feed_r.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,"[12669243, 2629939, 2707227, 12719360, 1261644..."
2,timezone,CET
3,start_date,20181106
4,end_date,20181111
5,num_routes,905
6,num_trips,118861
7,num_stops,6755
8,num_shapes,0
9,num_frequencies,0


In [6]:
feed_r.map_stops(feed_r.stops.stop_id)

### Simplify services

Once the feed is restricted on a few dates only, it is possible to simplify its services by grouping those operating on the same dates.  
/!\ This makes GTFS edition more complicated afterwards as different trips will run on the same services!

In [7]:
feed_r.group_services()

In [8]:
feed_r.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,"[12672285, 12719373, 2699953]"
2,timezone,CET
3,start_date,20181106
4,end_date,20181111
5,num_routes,905
6,num_trips,118861
7,num_stops,6755
8,num_shapes,0
9,num_frequencies,0


In [9]:
feed_r.calendar

In [10]:
feed_r.calendar_dates

,service_id,date,exception_type
0,12672285,20181106,1
471,2699953,20181106,1
730,12719373,20181111,1
1134,2699953,20181111,1


Only 3 services remain: one operating on 20181106, one operating on 20181111 and one operating on both.

In [11]:
feed_r.map_stops(feed_r.stops.stop_id)

### Restrict to a given service

In [12]:
s_id = feed_r.calendar_dates[(feed_r.calendar_dates['date']=='20181111')].iloc[0]['service_id']
s_id

'12719373'

In [13]:
feed_s = feed_r.restrict(
    service_ids=[s_id]
)

In [14]:
feed_s.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,[12719373]
2,timezone,CET
3,start_date,20181111
4,end_date,20181111
5,num_routes,655
6,num_trips,42172
7,num_stops,3425
8,num_shapes,0
9,num_frequencies,0


In [15]:
feed_s.map_stops(feed_s.stops.stop_id)

### Restrict to a given time range

In [16]:
time_range = ['06:00:00', '09:00:00']  # time format must be HH:MM:SS
feed_t = feed_s.restrict(time_range=time_range)

In [17]:
feed_t.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,[12719373]
2,timezone,CET
3,start_date,20181111
4,end_date,20181111
5,num_routes,13
6,num_trips,13
7,num_stops,299
8,num_shapes,0
9,num_frequencies,0


In [18]:
feed_t.map_stops(feed_t.stops.stop_id)